# Modeling USDA ERS Food data

In [1]:
dataset_id = "USDA_ERS_modeled"

In [1]:
!bq --location=US mk --dataset {dataset_id}  #Note: This will not work if you already have a dataset with this name

Dataset 'responsive-cab-267123:USDA_ERS_modeled' successfully created.


The `FIPS_Market_Group` and `State_Codes` tables do not contribute anything to our planned analysis, as descriped in the DATASETS.txt file, so we don't transfer them over to our modeled dataset. The `Geo_Market_Group` table contains the same data as `Geo_Market`, so we can drop it.

To model:

* some child tables have the "name" instead of "id" as the FK. We think id should be FK 
* Drop the extra blank column(s) in `Market_Groups` table(s)
    * change column names to what's on ERD
* Add meaningul column names to `Geo_Market`: (check ERD)
    * the `nielson_name` (4th column) field in the `Geo_Master` table violates the 1NF design principle --drop it
* union `Food_#_Market` tables, add a column to represent the food# (This table needs a PK)
    * changed some attribute names in Food_#_Market (but feel free to change them back and fix ERD)
        * se to standard_error
        * n to sample_size
        * aggweight to agg_weight
        * totexp to tot_q_exp
    * deleted region and division attributes from the new unioned `Food_Market` because it's repetitive (can be traced back to Geo_Market parent table)
* anything else you can think of
* anything not transformed/modeled, add to TRANSFORMS.txt file
    * we can make the mapping table between primary and secondary dataset in the next milestone

### Copy Foods table from staging dataset

In [43]:
%%bigquery
create table USDA_ERS_modeled.Foods as select distinct * from USDA_ERS_staging.Foods

""


### Copy Food_Categories table from staging dataset

In [34]:
%%bigquery
create table USDA_ERS_modeled.Food_Categories as select distinct * from USDA_ERS_staging.Food_Categories

""


### Create the Market_Groups table that will contain market_id and market_names

We are dropping the three string fields from the staging dataset (only need market_id and market_name)

In [16]:
%%bigquery
create table USDA_ERS_modeled.Market_Groups as select market_id, market_name from USDA_ERS_staging.Market_Groups

Executing query with job ID: 22e28f41-7433-494e-b585-2bd76b4f0ae5
Query executing: 0.23s


ERROR:
 409 GET https://bigquery.googleapis.com/bigquery/v2/projects/responsive-cab-267123/queries/22e28f41-7433-494e-b585-2bd76b4f0ae5?timeoutMs=400&location=US&maxResults=0: Already Exists: Table responsive-cab-267123:USDA_ERS_modeled.Market_Groups

(job ID: 22e28f41-7433-494e-b585-2bd76b4f0ae5)

                                           -----Query Job SQL Follows-----                                           

    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |
   1:create table USDA_ERS_modeled.Market_Groups as select market_id, market_name from USDA_ERS_staging.Market_Groups
    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |


### Create Geo_Regions table (copy from staging dataset)

In [37]:
%%bigquery
create table USDA_ERS_modeled.Geo_Regions as select distinct * from USDA_ERS_staging.Geo_Regions

""


### Create Geo_Divisions table (copy from staging dataset)

In [45]:
%%bigquery
create table USDA_ERS_modeled.Geo_Divisions as select distinct * from USDA_ERS_staging.Geo_Divisions

""


### Create a table that combines all Food_#_Market Tables from staging dataset

Will use marketgroup (change name to market_id),
year, quarter, price, se (will explicitly name it as standard_error),
n (sample_size), aggweight (agg_weight), totexp (tot_q_exp),
and food_id (this will correspond to the original number specified in each staging table -> 1-54).

In [2]:
%%bigquery
create table USDA_ERS_modeled.Food_Market as 
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 1 as food_id from USDA_ERS_staging.Food_1_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 2 as food_id from USDA_ERS_staging.Food_2_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 3 as food_id from USDA_ERS_staging.Food_3_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 4 as food_id from USDA_ERS_staging.Food_4_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 5 as food_id from USDA_ERS_staging.Food_5_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 6 as food_id from USDA_ERS_staging.Food_6_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 7 as food_id from USDA_ERS_staging.Food_7_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 8 as food_id from USDA_ERS_staging.Food_8_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 9 as food_id from USDA_ERS_staging.Food_9_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 10 as food_id from USDA_ERS_staging.Food_10_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 11 as food_id from USDA_ERS_staging.Food_11_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 12 as food_id from USDA_ERS_staging.Food_12_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 13 as food_id from USDA_ERS_staging.Food_13_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 14 as food_id from USDA_ERS_staging.Food_14_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 15 as food_id from USDA_ERS_staging.Food_15_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 16 as food_id from USDA_ERS_staging.Food_16_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 17 as food_id from USDA_ERS_staging.Food_17_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 18 as food_id from USDA_ERS_staging.Food_18_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 19 as food_id from USDA_ERS_staging.Food_19_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 20 as food_id from USDA_ERS_staging.Food_20_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 21 as food_id from USDA_ERS_staging.Food_21_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 22 as food_id from USDA_ERS_staging.Food_22_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 23 as food_id from USDA_ERS_staging.Food_23_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 24 as food_id from USDA_ERS_staging.Food_24_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 25 as food_id from USDA_ERS_staging.Food_25_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 26 as food_id from USDA_ERS_staging.Food_26_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 27 as food_id from USDA_ERS_staging.Food_27_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 28 as food_id from USDA_ERS_staging.Food_28_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 29 as food_id from USDA_ERS_staging.Food_29_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 30 as food_id from USDA_ERS_staging.Food_30_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 31 as food_id from USDA_ERS_staging.Food_31_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 32 as food_id from USDA_ERS_staging.Food_32_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 33 as food_id from USDA_ERS_staging.Food_33_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 34 as food_id from USDA_ERS_staging.Food_34_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 35 as food_id from USDA_ERS_staging.Food_35_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 36 as food_id from USDA_ERS_staging.Food_36_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 37 as food_id from USDA_ERS_staging.Food_37_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 38 as food_id from USDA_ERS_staging.Food_38_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 39 as food_id from USDA_ERS_staging.Food_39_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 40 as food_id from USDA_ERS_staging.Food_40_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 41 as food_id from USDA_ERS_staging.Food_41_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 42 as food_id from USDA_ERS_staging.Food_42_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 43 as food_id from USDA_ERS_staging.Food_43_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 44 as food_id from USDA_ERS_staging.Food_44_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 45 as food_id from USDA_ERS_staging.Food_45_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 46 as food_id from USDA_ERS_staging.Food_46_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 47 as food_id from USDA_ERS_staging.Food_47_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 48 as food_id from USDA_ERS_staging.Food_48_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 49 as food_id from USDA_ERS_staging.Food_49_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 50 as food_id from USDA_ERS_staging.Food_50_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 51 as food_id from USDA_ERS_staging.Food_51_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 52 as food_id from USDA_ERS_staging.Food_52_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 53 as food_id from USDA_ERS_staging.Food_53_Market UNION DISTINCT
select GENERATE_UUID() as food_market_id, marketgroup as market_id, year, quarter, price, se as standard_error, n as sample_size, aggweight as agg_weight, totexp as tot_q_exp, 54 as food_id from USDA_ERS_staging.Food_54_Market;

""


### Create Geo_Market Table 

In [53]:
%%bigquery
create table USDA_ERS_modeled.Geo_Market as select distinct Marketgroup as market_id, Region as region_id, Division as division_id from USDA_ERS_staging.Geo_Market_Group

""


### Create Product_Food_Map Table 

In [12]:
%%bigquery
CREATE TABLE Product_Food_Map


Executing query with job ID: 06622ff0-6637-47b2-8b3d-06a318215a36
Query executing: 0.28s


ERROR:
 400 Syntax error: Expected ")" or "," but got "(" at [6:5]

(job ID: 06622ff0-6637-47b2-8b3d-06a318215a36)

                           -----Query Job SQL Follows-----                           

    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |
   1:CREATE TABLE Product_Food_Map
   2:(
   3:    StudentID int NOT NULL,
   4:    ClassroomID int NOT NULL,
   5:    CONSTRAINT PK_products PRIMARY KEY
   6:    (
   7:        product_id
   8:        food_id
   9:    ),
  10:    FOREIGN KEY (product_id) REFERENCES instacart_modeled.Products (product_id),
  11:    FOREIGN KEY (food_id) REFERENCES USDA_ERS_modeled.Food_Market (food_id)
  12:)
    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |


## Check Primary Keys

## Food_Market Table

food_market_id

In [7]:
%%bigquery
select count(*) as no_entries from USDA_ERS_modeled.Food_Market

,no_entries
0,57731


In [8]:
%%bigquery
select count(distinct food_market_id) as no_PKs from USDA_ERS_modeled.Food_Market

,no_PKs
0,57731


## Foods Table

In [49]:
%%bigquery
select count(*) as no_entries from USDA_ERS_modeled.Foods

,no_entries
0,54


In [50]:
%%bigquery
select count(distinct food_id) as no_PKs from USDA_ERS_modeled.Foods

,no_PKs
0,54


## Food_Categories Table

In [35]:
%%bigquery
select count(*) as no_entries from USDA_ERS_modeled.Food_Categories

,no_entries
0,4


In [36]:
%%bigquery
select count(distinct category_id) as no_PKs from USDA_ERS_modeled.Food_Categories

,no_PKs
0,4


## Market_Groups Table

In [22]:
%%bigquery
select count(*) as no_entries from USDA_ERS_modeled.Market_Groups

,no_entries
0,39


In [28]:
%%bigquery
select count(distinct market_id) as no_PKs from USDA_ERS_modeled.Market_Groups

,no_PKs
0,39


## Geo_Regions Table

In [56]:
%%bigquery
select count(*) as no_entries from USDA_ERS_modeled.Geo_Regions

,no_entries
0,4


In [57]:
%%bigquery
select count(distinct region_id) as no_entries from USDA_ERS_modeled.Geo_Regions

,no_entries
0,4


## Geo_Divisions Table

In [58]:
%%bigquery
select count(*) as no_entries from USDA_ERS_modeled.Geo_Divisions

,no_entries
0,9


In [59]:
%%bigquery
select count(distinct division_id) as no_PKs from USDA_ERS_modeled.Geo_Divisions

,no_PKs
0,9


## Geo_Market Table

In [62]:
%%bigquery
select count(*) as no_entries from USDA_ERS_modeled.Geo_Market

,no_entries
0,35


In [63]:
%%bigquery
select count(distinct market_id) as no_entries from USDA_ERS_modeled.Geo_Market

,no_entries
0,35


# Foreign Key Check

market_id

In [79]:
%%bigquery
select g.market_id
from USDA_ERS_modeled.Food_Market fm left join USDA_ERS_modeled.Market_Groups g
on fm.market_id= g.market_id left join USDA_ERS_modeled.Geo_Market gm on gm.market_id = g.market_id
where fm.market_id is null

,market_id


food_id

In [78]:
%%bigquery
select fm.food_id
from USDA_ERS_modeled.Food_Market fm left join USDA_ERS_modeled.Foods f
on fm.food_id= f.food_id 
where f.food_id is null

,food_id


food_category

In [72]:
%%bigquery
select f.food_category
from USDA_ERS_modeled.Food_Categories fc left join USDA_ERS_modeled.Foods f
on fc.category_id= f.food_category
where fc.category_id is null

,food_category


region_id

In [74]:
%%bigquery
select gr.region_id
from USDA_ERS_modeled.Geo_Regions gr left join USDA_ERS_modeled.Geo_Market gm
on gr.region_id= gm.region_id
where gm.region_id is null

,region_id


division_id

In [77]:
%%bigquery
select gm.division_id
from USDA_ERS_modeled.Geo_Divisions gd left join USDA_ERS_modeled.Geo_Market gm
on gd.division_id= gm.division_id
where gd.division_id is null

,division_id
